In [13]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gb-vpp-whoppity-dub-dub/mean_submission.csv
/kaggle/input/gb-vpp-whoppity-dub-dub/Bidirect_LSTM_model_6C.h5
/kaggle/input/gb-vpp-whoppity-dub-dub/Google_Brain_Keras_Model.png
/kaggle/input/gb-vpp-whoppity-dub-dub/Bidirect_LSTM_model_2C.h5
/kaggle/input/gb-vpp-whoppity-dub-dub/__results__.html
/kaggle/input/gb-vpp-whoppity-dub-dub/Bidirect_LSTM_model_3C.h5
/kaggle/input/gb-vpp-whoppity-dub-dub/Bidirect_LSTM_model_1C.h5
/kaggle/input/gb-vpp-whoppity-dub-dub/Bidirect_LSTM_model_7C.h5
/kaggle/input/gb-vpp-whoppity-dub-dub/Bidirect_LSTM_model_5C.h5
/kaggle/input/gb-vpp-whoppity-dub-dub/__resultx__.html
/kaggle/input/gb-vpp-whoppity-dub-dub/median_submission.csv
/kaggle/input/gb-vpp-whoppity-dub-dub/Bidirect_LSTM_model_4C.h5
/kaggle/input/gb-vpp-whoppity-dub-dub/__notebook__.ipynb
/kaggle/input/gb-vpp-whoppity-dub-dub/__output__.json
/kaggle/input/gb-vpp-whoppity-dub-dub/custom.css
/kaggle/input/gb-vpp-whoppity-dub-dub/__results___files/__results___15_0.png
/kaggle/input/ventil

In [14]:
train = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/train.csv')
targets = train['pressure']
targets

0           5.837492
1           5.907794
2           7.876254
3          11.742872
4          12.234987
             ...    
6035995     3.869032
6035996     3.869032
6035997     3.798729
6035998     4.079938
6035999     3.869032
Name: pressure, Length: 6036000, dtype: float64

In [15]:
test = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/test.csv')
test.shape

(4024000, 7)

In [16]:
sub = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/sample_submission.csv')
sub.shape

(4024000, 2)

In [17]:
median_sub = pd.read_csv('../input/gb-vpp-whoppity-dub-dub/median_submission.csv')
median_sub.head()

,id,pressure
0,1,6.259344
1,2,5.978134
2,3,7.102974
3,4,7.595091
4,5,9.141746


In [18]:
targets = train['pressure']
train.drop(['pressure', 'id'], axis=1, inplace=True)
test = test.drop(['id'], axis=1)

In [19]:
pseudo_l = median_sub['pressure']

In [20]:
test.shape

(4024000, 6)

In [21]:
train.shape

(6036000, 6)

In [22]:
targets.values

array([5.83749171, 5.90779385, 7.87625392, ..., 3.79872949, 4.07993807,
       3.86903163])

In [23]:
import lightgbm as lgb

lr = 0.2
p_m = 0
new_y = 0


for w in range(3):
    
    params_loss_1 = {
                    'n_estimators': 5000,
                    'learning_rate': 0.1}
    
    
    lgbm_reg_1 = lgb.LGBMRegressor(
                                **params_loss_1, 
                                objective='mae',
                                metric='mae',
                                n_jobs=-1, random_state = 2021, verbose = 0
                                )
    
    lgbm_reg_1.fit(test, pseudo_l)

    p_m = lgbm_reg_1.predict(train)

    new_y = targets.values - p_m
    
    
    
    params_loss_2 = {
                    'n_estimators': 5000,
                    'learning_rate': 0.1}
    
    
    lgbm_reg_2 = lgb.LGBMRegressor(
                                **params_loss_2, 
                                objective='mae',
                                metric='mae',
                                n_jobs=-1, verbose = 0, random_state = 2020
                                )
    
    lgbm_reg_2.fit(
                train, 
                new_y
                )

    p_a = lgbm_reg_2.predict(test)

    my_y = median_sub['pressure'].values + (p_a * lr)


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.053825 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.080264 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.056535 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.077787 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing

In [24]:
my_y

array([6.24259954, 5.95034736, 7.10800514, ..., 6.45126386, 6.17877995,
       6.31820115])

In [25]:
from sklearn.metrics import mean_absolute_error

print(mean_absolute_error(targets, p_m))

1.9031754651181227


In [31]:
print(mean_absolute_error(median_sub['pressure'].values, my_y))

0.050676802484954354


In [33]:
train_1 = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/train.csv')
all_pressure = np.sort(train_1.pressure.unique())
print('The first 25 unique pressures...')
PRESSURE_MIN = all_pressure[0].item()
PRESSURE_MAX = all_pressure[-1].item()
all_pressure[:25]

The first 25 unique pressures...


array([-1.89574429, -1.82544215, -1.75514   , -1.68483786, -1.61453571,
       -1.54423357, -1.47393142, -1.40362928, -1.33332713, -1.26302499,
       -1.19272284, -1.12242069, -1.05211855, -0.9818164 , -0.91151426,
       -0.84121211, -0.77090997, -0.70060782, -0.63030568, -0.56000353,
       -0.48970139, -0.41939924, -0.34909709, -0.27879495, -0.2084928 ])

In [35]:
PRESSURE_STEP = ( all_pressure[1] - all_pressure[0] ).item()
all_pressure[1:26] - all_pressure[:25]

array([0.07030215, 0.07030215, 0.07030215, 0.07030215, 0.07030215,
       0.07030215, 0.07030215, 0.07030215, 0.07030215, 0.07030215,
       0.07030215, 0.07030215, 0.07030215, 0.07030215, 0.07030215,
       0.07030215, 0.07030215, 0.07030215, 0.07030215, 0.07030215,
       0.07030215, 0.07030215, 0.07030215, 0.07030215, 0.07030215])

In [36]:
median_sub['pressure'] = my_y
median_sub["pressure"] =\
    np.round( (median_sub.pressure - PRESSURE_MIN)/PRESSURE_STEP ) * PRESSURE_STEP + PRESSURE_MIN
median_sub.pressure = np.clip(median_sub.pressure, PRESSURE_MIN, PRESSURE_MAX)
median_sub.to_csv('submission.csv', index=False)